# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution. 

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [53]:
def check_for_sex(sex):
    if not isinstance(sex, str):
        return None
    sex = sex.lower()
    if sex == "male":
        return 1
    elif sex == "female":
        return 0
    else:
        return None
        
def check_for_age(age):
    try:
        age = int(age)
    except ValueError:
        return None
    if 16 <= age <= 49:
        return 0
    elif 50 <= age <= 65:
        return 2
    elif 66 <= age <= 80:
        return 3
    elif age > 80:
        return 4
    else:
        return None

def check_for_rr_bpm(rr_bpm):
    try:
        rr_bpm = float(rr_bpm)
    except ValueError:
        return None
    if rr_bpm < 9.0:
        return 3
    elif 9.0 <= rr_bpm <= 11.0:
        return 1
    elif 12.0 <= rr_bpm <= 20.0:
        return 0
    elif 21.0 <= rr_bpm <= 24.0:
        return 2
    elif rr_bpm > 24.0:
        return 3
    else:
        return None

def check_for_sp2o2(sp2o2):
    try:
        sp2o2 = float(sp2o2)
    except ValueError:
        return None
    if sp2o2 > .95:
        return 0
    elif .94 <= sp2o2 <= .95:
        return 1
    elif .92 <= sp2o2 < .94:
        return 2
    elif sp2o2 < .92:
        return 3
    else:
        return None 

def check_for_hr_bpm(hr_bpm):
    try:
        hr_bpm = float(hr_bpm)
    except ValueError:
        return None
    if hr_bpm < 41.0:
        return 3
    elif 41.0 <= hr_bpm <= 50.0:
        return 1
    elif 51.0 <= hr_bpm <= 90.0:
        return 0
    elif 91.0 <= hr_bpm <= 110.0:
        return 1
    elif 111.0 <= hr_bpm <= 130.0:
        return 2
    elif hr_bpm > 130.0:
        return 3
    else:
        return None 
    
def check_for_s_bp(s_bp):
    try:
        s_bp = float(s_bp)
    except ValueError:
        return None
    if s_bp < 91.0:
        return 3
    elif 91.0 <= s_bp <= 100.0:
        return 2
    elif 101.0 <= s_bp <= 110.0:
        return 1
    elif 111.0 <= s_bp <= 219.0:
        return 0
    elif s_bp > 219.0:
        return 3
    else:
        return None  
    
def check_for_temp(temp):
    try:
        temp = float(temp)
    except ValueError:
        return None
    if temp < 35.1:
        return 3
    elif 35.1 <= temp <= 36.0:
        return 1
    elif 36.1 <= temp <= 38.0:
        return 0
    elif 38.1 <= temp <= 39.0:
        return 1
    elif temp > 39.0:
        return 2
    else:
        return None 
    
def check_for_alertness(alertness):
    if not isinstance(alertness, str):
        return None
    alertness = alertness.lower().strip()
    if alertness == 'alert':
        return 0
    elif alertness == 'confused or not alert':
        return 3
    else:
        return None

def check_for_ins_o2(ins_o2):
    if not isinstance(ins_o2, str):
        return None
    ins_o2 = ins_o2.lower().strip()
    if ins_o2 == 'air':
        return 0
    elif ins_o2 == 'supplemental oxygen':
        return 2
    else:
        return None

def check_for_perf_stat(perf_stat):
    if not isinstance(perf_stat, str):
        return None
    perf_stat = perf_stat.lower().strip()
    if perf_stat == 'unrestricted normal activity':
        return 0
    elif perf_stat == 'limited strenuous activity, can do light activity':
        return 1
    elif perf_stat == 'limited activity, can self-care':
        return 2
    elif perf_stat == 'limited self-care':
        return 3
    elif perf_stat == 'bed/chair bound, no self-care':
        return 4
    else:
        return None

def get_30_day_probability(points):
    if points == 0 or points == 1:
        return 0.01
    elif points == 2 or points == 3:
        return 0.02
    elif points == 4:
        return 0.03
    elif points == 5:
        return 0.09
    elif points == 6:
        return 0.15
    elif points == 7:
        return 0.18
    elif points == 8:
        return 0.22
    elif points == 9:
        return 0.26
    elif points == 10:
        return 0.29
    elif points == 11:
        return 0.34
    elif points == 12:
        return 0.38
    elif points == 13:
        return 0.46
    elif points == 14:
        return 0.47
    elif points == 15:
        return 0.49
    elif points == 16:
        return 0.55
    elif points >= 17 and points <= 25:
        return 0.59
    elif points >= 26:
        return 0.99
    else:
        return None


def priest(sex: str, age: str, rr_bpm: str, sp2o2: str, hr_bpm: str, s_bp: str, temp: str, alertness: str, ins_o2: str, perf_stat: str) -> float|None:
    """
    logic as per mdcalc.com

    Args:
        sex (str): Sex
        age (str): Age, years
        rr_bpm (str): Respiratory rate, breaths per min
        sp2o2 (str): Oxygen saturation
        hr_bpm (str): Heart rate, beats per min
        s_bp (str): Systolic BP, mmHg
        temp (str): Temperature
        alertness (str): Alertness
        ins_o2 (str): Inspired oxygenInspired oxygen
        perf_stat (str): Performance status

    Returns:
        float: risk 
        None: in case of invalid input
        
    >>> print(priest('male', '45', '20.0', '.96', '95.0', '120.0', '37.5', 'Alert', 'air', 'unrestricted normal activity'))
    0.02

    >>> print(priest('female', '70', '25.0', '.91', '75.0', '110.0', '36.8', 'Confused or not alert', 'supplemental oxygen', 'limited activity, can self-care'))
    0.59

    >>> print(priest('other', '30', '15.0', '.93', '80.0', '100.0', '36.0', 'Something', 'supplemental oxygen', 'limited self-care'))
    None
    """
    total_points = 0
    # Check for sex
    sex_points = check_for_sex(sex)
    if sex_points is not None:
        total_points += sex_points
    else:
        return None
    
    # Check for age
    age_points = check_for_age(age)
    if age_points is not None:
        total_points += age_points
    else:
        return None
    
    # Check for rr_bpm
    rr_bpm_points = check_for_rr_bpm(rr_bpm)
    if rr_bpm_points is not None:
        total_points += rr_bpm_points
    else:
        return None
    
    sp2o2_points = check_for_sp2o2(sp2o2)
    if sp2o2_points is not None:
        total_points += sp2o2_points
    else:
        return None
    
    hr_bpm_points = check_for_hr_bpm(hr_bpm)
    if hr_bpm_points is not None:
        total_points += hr_bpm_points
    else:
        return None
    
    s_bp_points = check_for_s_bp(s_bp)
    if s_bp_points is not None:
        total_points += s_bp_points
    else:
        return None
    
    temp_points = check_for_temp(temp)
    if temp_points is not None:
        total_points += temp_points
    else:
        return None
    
    alertness_points = check_for_alertness(alertness)
    if alertness_points is not None:
        total_points += alertness_points
    else:
        return None
    
    ins_o2_points = check_for_ins_o2(ins_o2)
    if ins_o2_points is not None:
        total_points += ins_o2_points
    else:
        return None
    
    perf_stat_points = check_for_perf_stat(perf_stat)
    if perf_stat_points is not None:
        total_points += perf_stat_points
    else:
        return None
    
    return get_30_day_probability(total_points)
    

In [54]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    print(priest('male', '45', '20.0', '.96', '95.0', '120.0', '37.5', 'Alert', 'air', 'unrestricted normal activity'))
Expecting:
    0.02
ok
Trying:
    print(priest('female', '70', '25.0', '.91', '75.0', '110.0', '36.8', 'Confused or not alert', 'supplemental oxygen', 'limited activity, can self-care'))
Expecting:
    0.59
ok
Trying:
    print(priest('other', '30', '15.0', '.93', '80.0', '100.0', '36.0', 'Something', 'supplemental oxygen', 'limited self-care'))
Expecting:
    None
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [55]:
!pip install requests

DEPRECATION: Loading egg at c:\users\dell\appdata\local\programs\python\python311\lib\site-packages\scrape_linkedin-0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [56]:
import requests
def find_hospital(age: str, sex: str, risk: float) -> str|None:
    """
    Find a hospital based on age, sex, and risk percentage.

    Args:
        age (str): Age of the patient.
        sex (str): Sex of the patient ('male' or 'female').
        risk (float): Risk percentage.

    Returns:
        str: Name of the hospital if found.
        None: If no hospital is found or an error occurs during the API request.
    Raises:
        ValueError: If the provided age is not an integer or if the sex is not 'male' or 'female'.
        requests.exceptions.RequestException: If there's an error in making the HTTP request.

    Examples:
        >>> print(find_hospital('35', 'fEmale', 5.2))
        Emory Dunwoody Medical Center
        >>> print(find_hospital('40', 'Male', 0.1))
        Southwest Hospital and Medical Center
        >>> print(find_hospital('dd', 'male', 0.1))
        None
    """
    url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}"
    url = url.format(age=age, sex=sex.lower(), risk=risk)
    try:
        res = requests.get(url)
        if res.ok:
            json_response = res.json()
            return json_response.get("hospital", None)
    except:
        pass
    return None
    


In [57]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    print(find_hospital('35', 'fEmale', 5.2))
Expecting:
    Emory Dunwoody Medical Center
ok
Trying:
    print(find_hospital('40', 'Male', 0.1))
Expecting:
    Southwest Hospital and Medical Center
ok
Trying:
    print(find_hospital('dd', 'male', 0.1))
Expecting:
    None
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [58]:
# import requests
# import json
# url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"

# res = requests.get(url)
# with open("ga_hospitals.json", "w") as json_file:
#     json.dump(res.json(), json_file)

    

In [59]:
import json 
def get_address(hospital_name: str) -> str|None:
    """
    Get the address of a hospital based on its name.

    Args:
        hospital_name (str): Name of the hospital.

    Returns:
        str: Address of the hospital if found.
        None: If the hospital name is not found or an error occurs during the API request.

    Raises:
        requests.exceptions.RequestException: If there's an error in making the HTTP request.

    Examples:
        >>> print(get_address('COOK MEDICAL CENTER'))
        706 NORTH PARRISH AVENUE
        >>> print(get_address('BACON COUNTY HOSPITAL'))
        302 SOUTH WAYNE STREET
        >>> print(get_address(123))
        None
    """
    hospital_details = None
    with open("ga_hospitals.json") as json_file:
        hospital_details = json.load(json_file)
    
    if hospital_details is None:
        return None
    if not isinstance(hospital_name, str):
        return None
    shortlisted_hospital = hospital_details.get(hospital_name.upper(), None)
    if isinstance(shortlisted_hospital, dict):
        return shortlisted_hospital.get("ADDRESS", None)
    return None

In [60]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    print(get_address('COOK MEDICAL CENTER'))
Expecting:
    706 NORTH PARRISH AVENUE
ok
Trying:
    print(get_address('BACON COUNTY HOSPITAL'))
Expecting:
    302 SOUTH WAYNE STREET
ok
Trying:
    print(get_address(123))
Expecting:
    None
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [61]:
# import requests
# url = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"

# res = requests.get(url)
# with open("people.psv", "w") as psv_file:
#     psv_file.write(res.text)

    

In [62]:

import csv
def process_people(path="people.psv"):
    """
    Process patient details from a pipe-separated values (PSV) file, perform necessary computations and API calls,
    and output the results into a JSON file.

    Args:
        path (str, optional): The path to the PSV file containing patient details. Defaults to "people.psv".

    Returns:
        list: List of dictionaries containing patient details, hospital, and address information.
        None: If there is an error during the file reading or any other exception.

    """
    patient_details = None
    try:
        patient_details_with_header = list(csv.reader(open(path), delimiter='|'))
        patient_details = patient_details_with_header[1:]
    except:
        return None
    if patient_details is None:
        return None
    patient_records = {}
    for patient in patient_details:
        patient_id, sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status = patient
        risk = None
        hospital = None
        address = None
        try:
            risk = priest(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status)
            if risk is not None:
                hospital = find_hospital(age, sex, risk)
            else:
                hospital = None
            if hospital is not None:
                address = get_address(hospital)
            else:
                address = None
            age = int(age)
            breath = float(breath)
            o2sat = float(o2sat)
            heart = float(heart)
            systolic = float(systolic)
            temp = float(temp)
        except:
        
            pass
        finally:
            patient_records[patient_id] = [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, risk, hospital, address]
    import json
    with open("output_details.json", "w") as json_file:
        json.dump(patient_records, json_file)
    return patient_records
        
process_people()

{'E9559': ['FEMALE',
  40,
  24.0,
  0.96,
  105.0,
  115.0,
  34.9,
  'ALERT',
  'AIR',
  'unrestricted normal activity',
  0.15,
  'Select Specialty Hospital - Northeast Atlanta',
  '1821 CLIFTON ROAD NE'],
 'E9385': ['Female',
  51,
  19.0,
  0.99,
  90.0,
  91.0,
  38.1,
  'ALERT',
  'supplemental oxygen',
  'unrestricted normal activity',
  0.18,
  'Select Specialty Hospital - Northeast Atlanta',
  '1821 CLIFTON ROAD NE'],
 'E3067': ['female',
  40,
  29.0,
  0.96,
  105.0,
  95.0,
  38.1,
  'ALERT',
  'air',
  'unrestricted normal activity',
  0.18,
  'Select Specialty Hospital - Northeast Atlanta',
  '1821 CLIFTON ROAD NE'],
 'E9422': ['FEMALE',
  66,
  19.0,
  0.96,
  135.0,
  115.0,
  38.1,
  'ALERT',
  'AIR',
  'bed/chair bound, no self-care',
  0.34,
  'Wesley Woods Geriatric Hospital',
  '1821 CLIFTON ROAD, NE'],
 'E8661': ['MALE',
  18,
  22.0,
  0.95,
  105.0,
  115.0,
  36.3,
  'Confused or Not Alert',
  'air',
  'unrestricted normal activity',
  0.22,
  'Southwest Hospi

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [63]:
# import requests 
# url = 'https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5'
# res = requests.get(url)
# import json
# with open("people_results.json", "w") as json_file:
#     json.dump(res.json(), json_file)

In [65]:
import json
people_results = json.load(open("people_results.json"))
calculated_output = json.load(open("output_details.json"))
people_results == calculated_output

True

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---